In [1]:
struct Mystruct
  num::Integer
  str::String
end

In [2]:
m = Mystruct(13, "hello")

Mystruct(13, "hello")

In [3]:
m.num

13

In [4]:
m.str

"hello"

In [5]:
fieldnames(Mystruct)

(:num, :str)

In [6]:
m.str = "goodbye"

ErrorException: setfield!: immutable struct of type Mystruct cannot be changed

In [7]:
mutable struct MutableStruct
  a::Float64
  b::Integer
end

In [8]:
s = MutableStruct(1,2)

MutableStruct(1.0, 2)

In [9]:
s.a=4.5

4.5

In [10]:
s.b = 7.1

InexactError: InexactError: Int64(7.1)

In [1]:
ranks = ['A','2','3','4','5','6','7','8','9','T','J','Q','K']
suits = ['\u2660','\u2661','\u2662','\u2663']

4-element Vector{Char}:
 '♠': Unicode U+2660 (category So: Symbol, other)
 '♡': Unicode U+2661 (category So: Symbol, other)
 '♢': Unicode U+2662 (category So: Symbol, other)
 '♣': Unicode U+2663 (category So: Symbol, other)

In [12]:
struct Card
  rank::Integer
  suit::Integer
end

In [4]:
c = Card(3,2)

3♡

In [16]:
c.rank

3

In [17]:
c.suit

2

In [3]:
Base.show(io::IO, c::Card) = print(io, "$(ranks[c.rank])$(suits[c.suit])")

In [5]:
c

3♡

In [20]:
Card(-10,78)

BoundsError: BoundsError: attempt to access 13-element Vector{Char} at index [-10]

In [2]:
struct Card
  rank::Int
  suit::Int

  # construct a card based on the rank and suit
  function Card(r::Int,s::Int)
    1 <= r <=13  || throw(ArgumentError("The rank must be an integer between 1 and 13."))
    1 <= s <= 4  || throw(ArgumentError("The suit must be an integer between 1 and 4."))
    new(r,s)
  end

  # construct a card based on the number in a deck
  function Card(i::Int)
    1 <= i <= 52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
    mod(i,13)==0 ? new(13,div(i,13)) : new(mod(i,13),div(i,13)+1)
  end
end

In [8]:
Card(-10,12)

ArgumentError: ArgumentError: The rank must be an integer between 1 and 13.

In [9]:
Card(3,2)

3♡

In [11]:
Card(42)

3♣

In [9]:
struct Hand
  cards::Vector{Card}
end

In [10]:
Base.show(io::IO,h::Hand) = print(io, "[$(join(h.cards,", "))]")

In [11]:
h1=Hand([Card(2,3),Card(12,1),Card(10,1),Card(10,4),Card(5,2)])

[2♢, Q♠, T♠, T♣, 5♡]

In [1]:
struct Polynomial{T <: Number}
  coeffs::Vector{T}
end

In [2]:
poly1=Polynomial([1,2,3])
poly2=Polynomial([1.0,2.0,3.0])
poly3=Polynomial([2//3,3//4,5//8])
poly4=Polynomial([im,2+0im,3-2im,-im])
poly5=Polynomial([n for n=1:6])

Polynomial{Int64}([1, 2, 3, 4, 5, 6])

In [15]:
poly3

Polynomial{Rational{Int64}}(Rational{Int64}[2//3, 3//4, 5//8])

In [18]:
function Base.show(io::IO, p::Polynomial)
  print(io, reduce((str, n) -> "$str $(n==1 ? "" : "+") $(p.coeffs[n]) x^$(n-1)", 1:length(p.coeffs),init=""))
end

In [19]:
poly5

  1 x^0 + 2 x^1 + 3 x^2 + 4 x^3 + 5 x^4 + 6 x^5

In [20]:
poly3

  2//3 x^0 + 3//4 x^1 + 5//8 x^2

In [21]:
poly1+poly2

MethodError: MethodError: no method matching +(::Polynomial{Int64}, ::Polynomial{Float64})
The function `+` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  +(!Matched::BitMatrix, !Matched::LinearAlgebra.UniformScaling)
   @ LinearAlgebra ~/.julia/juliaup/julia-1.11.0-beta2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/LinearAlgebra/src/uniformscaling.jl:151
  +(!Matched::Bool, !Matched::Complex{Bool})
   @ Base complex.jl:308
  ...


In [22]:
import Base.+

In [23]:
function +(p1::Polynomial{T},p2::Polynomial{S}) where {T <: Number, S <: Number}
  Polynomial(p1.coeffs+p2.coeffs)
end

+ (generic function with 198 methods)

In [24]:
poly1+poly2

  2.0 x^0 + 4.0 x^1 + 6.0 x^2

In [33]:
function eval(poly::Polynomial{T},x::Number) where T <: Number
  reduce((val,i) -> val + poly.coeffs[i]*x^(i-1), 1:length(poly.coeffs))
end

eval (generic function with 2 methods)

In [34]:
eval(poly1,-2)

9

In [31]:
function eval(poly::Polynomial{T}, x::Number) where T <: Number
  reduce((val,c) -> x*val+c, reverse(poly.coeffs))
end

eval (generic function with 2 methods)

In [32]:
eval(poly1,-2)

9

In [35]:
struct Root
  root::Float64    #  approximate value of the root
  x_eps::Float64   #  estimate of the error in the x variable
  f_eps::Float64   #  function value at the root f(root)
  num_steps::Int   #  number of steps the method used
  converged::Bool  #  whether or not the stopping criterion was reached
  max_steps::Int   #  the maximum number of steps allowed
end

In [36]:
using ForwardDiff

In [37]:
function newton(f::Function, x0::Number)
  local dx = 1
  local x1 = x0
  local steps = 0
  while abs(dx) > 1e-6 && steps < 10
    dx = f(x1)/ForwardDiff.derivative(f,x1)
    x1 -= dx
    steps += 1
  end
  Root(x1, dx, f(x1), steps, steps < 10, 10)
end

newton (generic function with 1 method)

In [38]:
newton(x->x^2-2, 1)

Root(1.4142135623730951, 1.5947429102833119e-12, 4.440892098500626e-16, 5, true, 10)

In [39]:
function Base.show(io::IO,r::Root)
  str = r.converged ? """The root is approximately x̂ = $(r.root)
    An estimate for the error is $(r.x_eps)
    with f(x̂) = $(r.f_eps)
    which took $(r.num_steps) steps""" :
    """The root was not found within $(r.max_steps) steps.
    Currently, the root is approximately x̂ = $(r.root).
    An estimate for the error is $(r.x_eps)
    with f(x̂) = $(r.f_eps)."""
  print(io,str)
end

In [40]:
r = newton(x->x^2-2, 1)

The root is approximately x̂ = 1.4142135623730951
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.440892098500626e-16
which took 5 steps

In [12]:
newton(x->x^2+1,1.1)

The root was not found within 10 steps.
Currently, the root is approximately x̂ = 0.030421004027873844.
An estimate for the error is 1.0004626117382218
with f(x̂) = 1.0009254374860639.